In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_data.csv')
test = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/test_data.csv')
train_labels=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_labels.csv')
fitness=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/fitness_values.csv')


In [2]:
train=pd.merge(train,train_labels,left_on='patient_id',right_on='patient_id')

In [3]:
list=(['patient_id', 'event_name','specialty','plan_type'])
    
for x in train.columns:
        if x in list:
            train[x]=train[x].astype(np.object)
for x in test.columns:
        if x in list:
            test[x]=test[x].astype(np.object)

In [4]:
print(train.shape)
train.drop_duplicates(keep=False,inplace=True)
test.drop_duplicates(keep=False,inplace=True)

print(train.shape)
print(test.shape)

(14446880, 7)
(5242365, 7)
(2248964, 6)


In [ ]:
#Alternate code for gridsearch with simple test_trainsplit

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [500,1000,1500],
        'reg_lambda' : [1,3,15],
        'learning_rate' : [0.3,0.01,0.09],
        'depth' : [4,7,10],
        'bagging_temperature' : [0,1,8],
        }
                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

X_train,X_val,y_train,y_val = train_test_split(Xd,yd,test_size=0.25,random_state = 13)
categorical_features_indices = np.where(Xd.dtypes =='object')[0]
catb = CatBoostRegressor(X_train.values,y_train.values,cat_features=categorical_features_indices,random_seed=13)

#skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 13)

grid = GridSearchCV(estimator=catb, param_grid=params,cv=(X_val, y_val), scoring=roc_auc, n_jobs=4, verbose=5 )
#grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/BigMart3/Outputs/xgb-grid-search-results-Catreg_multipleoptions.csv', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

In [ ]:
#Alternate code for gridsearch with stratkfold

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [500,1000,1500],
        'reg_lambda' : [1,3,15],
        'learning_rate' : [0.3,0.01,0.09],
        'depth' : [4,7,10],
        'bagging_temperature' : [0,1,8],
        }
                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(Xd.dtypes =='object')[0]
catb = CatBoostRegressor(cat_features=categorical_features_indices,random_seed=13)

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 13)

grid = GridSearchCV(estimator=catb, param_grid=params, scoring=roc_auc, n_jobs=4, cv=skf.split(Xd,yd), verbose=5 )
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/BigMart3/Outputs/xgb-grid-search-results-Catreg_multipleoptions.csv', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

In [5]:
#Catboost without gridsearch

from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import KFold,train_test_split


X,y=train.drop('outcome_flag',axis=1),train.outcome_flag
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 13)

categorical_features_indices = np.where(X_train.dtypes =='object')[0]


err=[]
y_pred_tot=[]
#from sklearn.model_selection import KFold,StratifiedKFold
#fold=KFold(n_splits=5,shuffle=True,random_state=1994)
#for train_index, test_index in fold.split(X,y):
#    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#    y_train, y_test = y[train_index], y[test_index]
print('start')
cat_model5 = CatBoostRegressor(n_estimators=500, # use large n_estimators deliberately to make use of the early stopping
                         reg_lambda=15,
                         eval_metric='AUC',
                         random_seed=13,
                         learning_rate = 0.01,
                         depth = 7,
                         bagging_temperature = 1.0,cat_features=categorical_features_indices)
        
cat_model5.fit(X_train.values,y_train.values,eval_set=(X_val, y_val),plot=False,use_best_model=True,verbose_eval=50) # early stopping set to 100 to prevent overfitting
    #print('pred...')
    #p = cat_model5.predict(X_test)
    #if sum(p<0)>0:
    #    p=pos(p)
    #print("err: ",np.sqrt(mean_squared_error(y_test,p)))
    #err.append(np.sqrt(mean_squared_error(y_test,p)))
pred = cat_model5.predict(test)
y_pred_tot.append(pred)

start
0:	test: 1.0000000	best: 1.0000000 (0)	total: 1.38s	remaining: 11m 30s
50:	test: 1.0000000	best: 1.0000000 (0)	total: 57.5s	remaining: 8m 26s
100:	test: 1.0000000	best: 1.0000000 (0)	total: 1m 50s	remaining: 7m 14s
150:	test: 1.0000000	best: 1.0000000 (0)	total: 2m 42s	remaining: 6m 16s
200:	test: 1.0000000	best: 1.0000000 (0)	total: 3m 37s	remaining: 5m 23s
250:	test: 1.0000000	best: 1.0000000 (0)	total: 4m 40s	remaining: 4m 37s
300:	test: 1.0000000	best: 1.0000000 (0)	total: 5m 47s	remaining: 3m 49s
350:	test: 1.0000000	best: 1.0000000 (0)	total: 6m 44s	remaining: 2m 51s
400:	test: 1.0000000	best: 1.0000000 (0)	total: 7m 53s	remaining: 1m 56s
450:	test: 1.0000000	best: 1.0000000 (0)	total: 9m 2s	remaining: 59s
499:	test: 1.0000000	best: 1.0000000 (0)	total: 10m 27s	remaining: 0us

bestTest = 1
bestIteration = 0

Shrink model to first 1 iterations.


In [6]:
# y_pred=rf.predict(dftest_dum.drop('id',axis=1))
print(np.mean(err,0))
y_pred=np.mean(y_pred_tot,0)
print(y_pred)

s=pd.DataFrame({'patient_id':test.patient_id,'outcome_flag':y_pred})
s.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Catreg_trial_v0.1.csv',index=False)
s.head()

C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
[0.14396971 0.14396971 0.14396971 ... 0.14396971 0.14396971 0.14396971]


,patient_id,outcome_flag
47,patient_148,0.14397
49,patient_158,0.14397
52,patient_167,0.14397
59,patient_180,0.14397
69,patient_219,0.14397


In [7]:
s.describe()

,outcome_flag
count,2.248964e+06
mean,1.439697e-01
std,2.903234e-13
min,1.439697e-01
25%,1.439697e-01
50%,1.439697e-01
75%,1.439697e-01
max,1.439697e-01
